ZOMNA DE IMPORT

 ºPara poder usar pyspark en VScode tenemos que usar findspark.init()

In [2]:
import findspark
findspark.init()

In [3]:
from pyspark.sql.functions import expr, col
from pyspark.sql.types import StructField, StructType, StringType

In [4]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [5]:
from datetime import datetime, date
from pyspark.sql import Row

In [6]:
df = spark.createDataFrame([
    (1, 2., 'string1', date(2000, 1, 1), datetime(2000, 1, 1, 12, 0)),
    (2, 3., 'string2', date(2000, 2, 1), datetime(2000, 1, 2, 12, 0)),
    (3, 4., 'string3', date(2000, 3, 1), datetime(2000, 1, 3, 12, 0))
], schema='a long, b double, c string, d date, e timestamp')

In [7]:
df.show()

+---+---+-------+----------+-------------------+
|  a|  b|      c|         d|                  e|
+---+---+-------+----------+-------------------+
|  1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|  2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|  3|4.0|string3|2000-03-01|2000-01-03 12:00:00|
+---+---+-------+----------+-------------------+



In [8]:
df_data = spark.createDataFrame([
    ("Jorge", 10, 8, 5, None),
    ("Fito", 6, 7, 8, 10),
    ("Percy", None, None, None, 5)
],["Nombre", "Nota1", "Nota2", "Nota3", "Nota4"])

In [9]:
df_data.show()

+------+-----+-----+-----+-----+
|Nombre|Nota1|Nota2|Nota3|Nota4|
+------+-----+-----+-----+-----+
| Jorge|   10|    8|    5| null|
|  Fito|    6|    7|    8|   10|
| Percy| null| null| null|    5|
+------+-----+-----+-----+-----+



In [10]:
uiid = "ID_UIID"

In [11]:
lista_notas = [x for x in df_data.columns if "Nota" in x]
print(lista_notas)

['Nota1', 'Nota2', 'Nota3', 'Nota4']


In [23]:
df_info = df_data.withColumn("ID_UIID", expr("uuid()"))

In [24]:
df_select = df_info.select("ID_UIID", *lista_notas)
df_select.show()

+--------------------+-----+-----+-----+-----+
|             ID_UIID|Nota1|Nota2|Nota3|Nota4|
+--------------------+-----+-----+-----+-----+
|f00c9741-48d5-47a...|   10|    8|    5| null|
|3c65bf93-c465-463...|    6|    7|    8|   10|
|de05c527-4c85-4e0...| null| null| null|    5|
+--------------------+-----+-----+-----+-----+



In [31]:
suma = 0
contador = 0
media = 0
lista_medias_nostas = []

In [32]:
for rows_data in df_select.collect():
    id = rows_data[0]
    # print(f"ID = {id}")
    for x in range(1,len(rows_data)):
        # print(rows_data[x])
        notas = rows_data[x]
        if notas is not None:
            suma += notas
            contador += 1
    if suma > 0:
        media = (suma / contador)
        lista_medias_nostas.append([id, media])
    suma = 0
    contador = 0
    media = 0

In [33]:
print(lista_medias_nostas)

[['f00c9741-48d5-47a7-9674-67a990ec443d', 7.666666666666667], ['3c65bf93-c465-463c-9926-aee0ce01f3e3', 7.75], ['de05c527-4c85-4e00-8074-e591bc10832e', 5.0]]


In [34]:
df_media = spark.sparkContext.parallelize(lista_medias_nostas).toDF(["ID_UIID", "MET_NOTAS"])

In [35]:
df_media_final = df_info.join(df_media, df_info.ID_UIID == df_media.ID_UIID, 'left').drop("ID_UIID")

In [36]:
df_media_final.show()

+------+-----+-----+-----+-----+-----------------+
|Nombre|Nota1|Nota2|Nota3|Nota4|        MET_NOTAS|
+------+-----+-----+-----+-----+-----------------+
|  Fito|    6|    7|    8|   10|             7.75|
| Percy| null| null| null|    5|              5.0|
| Jorge|   10|    8|    5| null|7.666666666666667|
+------+-----+-----+-----+-----+-----------------+



In [37]:
df_media_final.printSchema()

root
 |-- Nombre: string (nullable = true)
 |-- Nota1: long (nullable = true)
 |-- Nota2: long (nullable = true)
 |-- Nota3: long (nullable = true)
 |-- Nota4: long (nullable = true)
 |-- MET_NOTAS: double (nullable = true)

